In [253]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import time

# Extract CSVs into DataFrames

### California_Drought_Data
    * Drop columns
        *Mapdate is a duplicate date of Start_date
        *StatisticsFormatID are all the number 1
        *FIPS is a Weather Service ID
        *State all are CA
     * Removed county from county names
     * Renamed columns with underscores for postgres

In [254]:
drought_file = 'Resources/California_Drought_Data.csv'
drought_df = pd.read_csv(drought_file)
drought_df.head()

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20200602,6001,Alameda County,CA,0.0,100.0,100.0,44.56,0.0,0.0,6/2/2020,6/8/2020,1
1,20200526,6001,Alameda County,CA,0.0,100.0,100.0,44.56,0.0,0.0,5/26/2020,6/1/2020,1
2,20200519,6001,Alameda County,CA,0.0,100.0,100.0,44.56,0.0,0.0,5/19/2020,5/25/2020,1
3,20200512,6001,Alameda County,CA,0.0,100.0,100.0,44.56,0.0,0.0,5/12/2020,5/18/2020,1
4,20200505,6001,Alameda County,CA,0.0,100.0,100.0,44.56,0.0,0.0,5/5/2020,5/11/2020,1


In [255]:
drought_df.describe()

,MapDate,FIPS,None,D0,D1,D2,D3,D4,StatisticFormatID
count,2.250400e+04,22504.000000,22504.000000,22504.000000,22504.000000,22504.000000,22504.000000,22504.000000,22504.0
mean,2.016292e+07,6058.000000,28.765133,71.234867,56.850556,45.261159,29.975341,16.912968,1.0
std,2.154171e+04,33.482082,43.576187,43.576187,47.745243,48.088729,43.841057,35.239000,0.0
min,2.013010e+07,6001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,2.014111e+07,6029.000000,0.000000,9.317500,0.000000,0.000000,0.000000,0.000000,1.0
50%,2.016092e+07,6058.000000,0.000000,100.000000,99.040000,0.590000,0.000000,0.000000,1.0
75%,2.018073e+07,6087.000000,90.682500,100.000000,100.000000,100.000000,91.910000,0.000000,1.0
max,2.020060e+07,6115.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.0


In [256]:
# Drop columns    
drought_df = drought_df.drop(columns=['MapDate','StatisticFormatID','FIPS', 'State'])

In [257]:
# Removed county from county names
drought_df['County'] = drought_df['County'].str.replace(r'County', '')

In [258]:
# Renamed columns with underscores for postgres
drought_df = drought_df.rename(columns={'ValidStart':'valid_start','ValidEnd':'valid_end', 'None':'no_drought', 'County':'county',
                                       'D0':'d0', 'D1':'d1', 'D2':'d2', 'D3':'d3', 'D4':'d4'})
drought_df.head()

,county,no_drought,d0,d1,d2,d3,d4,valid_start,valid_end
0,Alameda,0.0,100.0,100.0,44.56,0.0,0.0,6/2/2020,6/8/2020
1,Alameda,0.0,100.0,100.0,44.56,0.0,0.0,5/26/2020,6/1/2020
2,Alameda,0.0,100.0,100.0,44.56,0.0,0.0,5/19/2020,5/25/2020
3,Alameda,0.0,100.0,100.0,44.56,0.0,0.0,5/12/2020,5/18/2020
4,Alameda,0.0,100.0,100.0,44.56,0.0,0.0,5/5/2020,5/11/2020


In [259]:
drought_df.dtypes

county          object
no_drought     float64
d0             float64
d1             float64
d2             float64
d3             float64
d4             float64
valid_start     object
valid_end       object
dtype: object

In [260]:
counties = drought_df['county'].unique()
counties_df = pd.DataFrame(counties)
counties_df.head()

,0
0,Alameda
1,Alpine
2,Amador
3,Butte
4,Calaveras


### California_Fire_Data
    * Drop columns
        * CanonicalUrl
        * ConditionStatement
        * ControlStatement
        * StructuresEvacuated
        * UniqueId
        * Updated
        * Featured
        * Final
        * Public
        * Status
        * ArchiveYear
        * PercentContained
        * CalFireIncident
    * Use datetime and removed the timestamp and format 
    * Renamed columns with underscores for postgres

In [261]:
fire_file = "Resources/California_Fire_Data.csv"
fire_df = pd.read_csv(fire_file)
fire_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [262]:
fire_df.describe()

,AcresBurned,AirTankers,ArchiveYear,CrewsInvolved,Dozers,Engines,Fatalities,Helicopters,Injuries,Latitude,Longitude,PercentContained,PersonnelInvolved,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders
count,1633.000000,28.000000,1636.000000,171.000000,123.000000,191.000000,21.000000,84.000000,120.000000,1636.000000,1636.000000,1633.0,204.000000,67.000000,175.000000,0.0,30.000000,146.000000
mean,4589.443968,4.071429,2016.608802,11.561404,7.585366,23.565445,8.619048,5.357143,3.500000,37.203975,-108.082642,100.0,328.553922,67.970149,271.788571,NaN,522.800000,7.815068
std,27266.337722,6.399818,1.845340,14.455633,14.028616,41.004424,18.529642,7.265437,3.806231,135.401380,37.006927,0.0,521.138789,155.771975,1557.255963,NaN,739.586856,12.719251
min,0.000000,0.000000,2013.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-120.258000,-124.196290,100.0,0.000000,0.000000,0.000000,NaN,0.000000,1.000000
25%,35.000000,2.000000,2015.000000,2.500000,1.000000,5.000000,1.000000,1.000000,1.000000,34.165890,-121.768358,100.0,55.000000,1.000000,1.000000,NaN,0.000000,2.000000
50%,100.000000,2.000000,2017.000000,6.000000,2.000000,11.000000,3.000000,2.000000,3.000000,37.104065,-120.461560,100.0,151.500000,6.000000,7.000000,NaN,45.000000,4.000000
75%,422.000000,4.000000,2018.000000,13.500000,5.000000,24.000000,6.000000,5.000000,4.000000,39.086808,-117.474072,100.0,350.000000,49.500000,41.500000,NaN,1043.750000,6.000000
max,410203.000000,27.000000,2019.000000,82.000000,76.000000,256.000000,85.000000,29.000000,26.000000,5487.000000,118.908200,100.0,3100.000000,783.000000,18804.000000,NaN,2600.000000,79.000000


In [263]:
# Drop columns
fire_df = fire_df.drop(columns=['CanonicalUrl', 'ConditionStatement', 'ControlStatement', 'StructuresEvacuated',
                                'UniqueId', 'Updated','Featured', 'Final','Public', 'Status','ArchiveYear',
                                'PercentContained','CalFireIncident'])

In [264]:
# Changed to datetime and removed the timestamp
fire_df['Started'] = pd.to_datetime(fire_df['Started']).dt.strftime("%m/%d/%Y")
fire_df['Extinguished'] = pd.to_datetime(fire_df['Extinguished']).dt.strftime("%m/%d/%Y")

In [265]:
# Renamed columns with underscores for postgres & lowercase
fire_df = fire_df.rename(columns={'AcresBurned':'acres_burned','ArchiveYear':'archive_year', 'AdminUnit':'admin_unit',
                                'AirTankers':'air_tankers','CountyIds':'county_ids', 'CrewsInvolved':'crews_involved',
                                'FuelType':'fuel_type', 'MajorIncident':'major_incident', 'PersonnelInvolved':'personnel_involved',
                                'SearchDescription':'search_description', 'SearchKeywords':'search_keywords', 'StructuresDamaged':'structures_damaged',
                                'StructuresDestroyed':'structures_destroyed', 'StructuresThreatened':'structures_threatened', 'WaterTenders':'water_tenders', 'Active':'active', 'County':'county', 
                                'Dozers':'dozers', 'Engines':'engines', 'Extinguished':'extinguished', 'Fatalities':'fatalities', 'Helicopters':'helicopters', 
                                'Injuries':'injuries', 'Latitude':'latitude', 'Location':'location', 'Longitude':'longitude', 'Name':'name', 'Started':'started'})
fire_df = fire_df.rename(columns={'Counties':'county'})

In [266]:
fire_df.dtypes

acres_burned             float64
active                      bool
admin_unit                object
air_tankers              float64
county                    object
county_ids                object
crews_involved           float64
dozers                   float64
engines                  float64
extinguished              object
fatalities               float64
fuel_type                 object
helicopters              float64
injuries                 float64
latitude                 float64
location                  object
longitude                float64
major_incident              bool
name                      object
personnel_involved       float64
search_description        object
search_keywords           object
started                   object
structures_damaged       float64
structures_destroyed     float64
structures_threatened    float64
water_tenders            float64
dtype: object

In [267]:
keywords = fire_df['search_keywords'].unique()
search_keywords_df = pd.DataFrame(keywords)
search_keywords_df.head()

,0
0,"Rim Fire, Stanislaus National Forest, Yosemite..."
1,"Powerhouse Fire, May 2013, June 2013, Angeles ..."
2,"Mountain Fire, July 2013, Highway 243, Highway..."
3,"American Fire, August 2013, Deadwood Ridge, Fo..."
4,"Springs Fire, May 2013, Highway 101, Camarillo..."


# Create database connection

In [268]:
connection_string = "postgres:JPH401@mc@localhost:5432/cal_fires"
engine = create_engine(f'postgresql://{connection_string}')

In [269]:
# Confirm tables
engine.table_names()

[]

# Load DataFrames into database

In [270]:
counties_df.to_sql(name='counties', con=engine, if_exists='append', index=True)

In [271]:
drought_df.to_sql(name='drought', con=engine, if_exists='append', index=True)

In [272]:
fire_df.to_sql(name='fires', con=engine, if_exists='append', index=True)

In [273]:
search_keywords_df.to_sql(name='search_keywords', con=engine, if_exists='append', index=True)